# MCMC5.5: Exact Digonalization in Python

This is FYI. I am currently using Python 3.7.1. See also https://github.com/jhauschild/lecture_comp_methods/blob/master/2_exact_diagonalization/sol4_simple_ED.ipynb

In [43]:
import numpy as np
from scipy import sparse 
from itertools import chain
Lx = 2
Ly = 2
def spinoperators(sigma, i, N):
    if N < 1:
        return
    elif i == 1 and N == 1:
        return sigma
    elif N == 1:
        return sparse.csr_matrix(np.eye(2))
    elif i == N:
        return sparse.kron(spinoperators(sigma, i, N - 1), sigma, 'csr')
    else:
        return sparse.kron(spinoperators(sigma, i, N - 1), np.eye(2), 'csr')

sigma_x = sparse.csr_matrix(np.array([[0., 1.], [1., 0.]]))
sigma_y = sparse.csr_matrix(np.array([[0., -1.j], [1.j, 0.]]))
sigma_z = sparse.csr_matrix(np.array([[1., 0.], [0., -1.]]))
N = 2 * Lx * Ly
nnx = zip(range(0, N - 1, 2), range(1, N - 2, 2))
nny = chain.from_iterable(zip(range(2 * i, 2 * Lx * (Ly - 1)  + 1 + 2 * i, 2 * Lx), range(2 * i, 2 * Lx * (Ly - 1)  + 2 * i + 1, 2 * Lx)) for i in range(Lx))
nnz = zip(range(0, N - 1, 2), chain(range(2 * Lx + 1, N, 2), range(1, 2 * Lx, 2)))
list(nny) # need correction

[(0, 0), (4, 4), (2, 2), (6, 6)]